In [11]:
from transformers import  AutoTokenizer, AutoModelForSeq2SeqLM
from dimweb_persona_bot.dataloaders.seq2seq_samplers.seq2seq_samplers_hypothesis_2 import H2Seq2SeqInferencePersonaSampleV1
from dimweb_persona_bot.hyperparameters.causal_modeling_hyperparameters import H2PersonaChatHyperparametersV1
import torch

In [6]:
history = [1, 2, 3, 4, 5, 6]

[(history[i], history[i+1]) for i in range(0, len(history), 2) ]

[(1, 2), (3, 4), (5, 6)]

In [9]:
" | ".join([
            "i'm also a graduate student .",
            "i enjoy reading journals and guides related to psychology .",
            "my parents taught me survival skills .",
            "i walk dogs for a living .",
        ])

"i'm also a graduate student . | i enjoy reading journals and guides related to psychology . | my parents taught me survival skills . | i walk dogs for a living ."

In [8]:
"".split(" | ")

['']

In [12]:
model_path = "dim/persona_bot_2_28akcwik"
device = "cuda"

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

hyperparameters = H2PersonaChatHyperparametersV1(
	model_name="facebook/bart-base",
	model_architecture="seq2seq",
	chat_history_pair_length=7,
	persona_max_length=14, 
	chat_max_length=25,
)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [101]:
sample = H2Seq2SeqInferencePersonaSampleV1(
	tokenizer=tokenizer,
	hyperparameters=hyperparameters,
	dataset_sample={
		"persona": [
      		'I like chocolate ice cream.', 
        	"Sometimes I feel lonely.", 
        	"I like to play video games."
        ],
		"history": [
    	  	"Hi, do you like ice cream?", 
       		'i do like ice cream but i prefer chocolate ice cream',
			"I feel lonely",
			'i feel lonely when i play video games',
			"Do you wanna play with me? I think we can have fun together.",
			'of course. what else do you like?',
			"I like driving simulator games. What about you?",
			'i like driving simulator games too',
			"Let's play together.",
			'okay. what else do you like?',
			"When so you wanna play?",
    ]
	}
).get_sample()

for key in sample.keys():
    sample[key] = torch.tensor(sample[key]).unsqueeze(0).to(device)
    
# model(**sample)
answer = model.generate(**sample, 
	# do_sample=True, 
    # top_p=0.95,
    penalty_alpha=0.1, 
    top_k=10
)
tokenizer.batch_decode(answer, skip_special_tokens=True,)

['anytime. what is your favorite?']

In [108]:
from dimweb_persona_bot.inference.seq2seq_bots import DialogBotV2

bot2 = DialogBotV2(model=model,
    tokenizer=tokenizer,
    hyperparameters=hyperparameters,
    history=[
        'Hi, how are you doing?',
        "i'm doing well. how are you?",
        "I'm fixing a bug right now",
        "oh wow that's interesting. what bug are you fixing?",
	],
    persona=[
        "I'm a junior frontend developer.", 
        'I like racing games.', 
        'Sometimes I write code for fun.', 
        "I'm a computer science fresher."
	]
)

In [111]:
print("\n".join(["'Hi, how are you doing?'", "i'm doing well, how are you?", "I'm fixing a bug right now", "what bug? i'm a developer.", 'Button sometimes turns red, sometimes disappears', "that's interesting. i'm a computer science major.", 'Can you help me please with it?', 'sure. what do you do for fun?', "I like to play in Dirt2. It's racing game", "that's a very popular game.", 'Do you like it?', 'i do. it is very fun.']))

'Hi, how are you doing?'
i'm doing well, how are you?
I'm fixing a bug right now
what bug? i'm a developer.
Button sometimes turns red, sometimes disappears
that's interesting. i'm a computer science major.
Can you help me please with it?
sure. what do you do for fun?
I like to play in Dirt2. It's racing game
that's a very popular game.
Do you like it?
i do. it is very fun.


In [113]:
from dimweb_persona_bot.dataloaders.persona_chat_dataloaders import PersonaChatDatasetV1
dataset = PersonaChatDatasetV1(input_dataset_path="./datasets/persona_chat/valid.json")

dataset[104]

{'persona': ["i'm also a graduate student .",
  'i enjoy reading journals and guides related to psychology .',
  'my parents taught me survival skills .',
  'i walk dogs for a living .'],
 'history': ['hey how are you today',
  "i'm busy studying psychology for my graduate school class . you ?",
  'i have retired and now spend my time as a pro gambler',
  'sounds cool ! i could be your dog walker when you are busy .',
  'well being retired i have a lot of spare time either in a casino or outdoors',
  'i love the outdoors . i can survive in the wilderness for weeks',
  'yeah me to just got back from two weeks in the bush',
  'did your parents teach you to survive like mine did ?',
  'my dad did he has his own outdoor show its possible you have seen it',
  'i may have . what is it called ?',
  'jim shockey he has survived in places all over the world',
  'i love that show ! i watch it between studying .',
  "yeah didn't get to see him much as a child",
  'that must have been hard . my pa

In [109]:
response = bot2.single_chat(
	'That is cool. What do you do on this work?',
)
print(response)
print(bot2.persona)
print(bot2.history)

i'm a senior frontend developer.
['Sometimes I write code for fun.', "I'm a junior frontend developer.", "I'm a computer science fresher.", 'I like racing games.']
['Hi, how are you doing?', "i'm doing great. just got home from a long day of work.", 'Where do you work?', 'i work in the computer sciences department at the university of louisiana.']


In [53]:
bot2.history

['Hi, how are you doing?',
 "i'm doing well. how are you?",
 "I'am fixing a bug right now",
 "oh wow that's interesting. what bug are you fixing?",
 "i'm fixing a computer that needs fixed",
 "oh wow that's pretty cool. what do you do for fun?"]

In [35]:
bot2.start_chat()

Bot: i'm doing well. how are you?
Bot: oh wow that's interesting. what bug are you fixing?
Bot: i like red hot chilli peppers


In [36]:
bot2.history

['Hi, how are you doing?',
 "i'm doing well. how are you?",
 "I'am fixing a bug right now",
 "oh wow that's interesting. what bug are you fixing?",
 "Oh, It's disgusting bug. The button on website sometimes is black and sometimes is red. What do you think?",
 'i like red hot chilli peppers']

### russian bot

In [3]:
from dimweb_persona_bot.inference.seq2seq_bots import DialogBotV1, DialogBotV2

In [10]:
model_path = "./models/28akcwik/6/"
device = "cuda"

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

hyperparameters = H2PersonaChatHyperparametersV1(
    chat_history_pair_length=7,
    max_response_length=512,
    persona_max_length=14,
    chat_max_length=25,
)

NameError: name 'AutoModelForSeq2SeqLM' is not defined

In [4]:
bot2 = DialogBotV2(model=model,
    tokenizer=tokenizer,
    hyperparameters=hyperparameters,
    history=[
        
	],
    persona=[
        "Я студент 4 курса.",
        "Люблю иногда бухать по пятницам.",
        "У меня нет девушки.",
        "Люблю писать код.",
	]
)

In [5]:
bot2.start_chat()

Bot: Привет, я студент 4 курса. А ты?
Bot: А что ты делаешь в свободное время?
Bot: Я пишу код. А ты?
Bot: В свободное время я играю в футбол. А ты?
